<a href="https://colab.research.google.com/github/dsamithmendis/python-programming/blob/main/Copy_of_EEX3372_Mini_Project_423646209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cab Service System — Task 01 Mini Project EEX3372

Run each cell in order before `!python main.py`


In [ ]:
!python main.py

Added: Car ID:C001 cap:4 AC:True
Added: Car ID:C002 cap:3 AC:False
Added: Van ID:V001 cap:6 AC:True
Added: 3W ID:T001 cap:3
Added: Truck ID:TR1 size:7ft
Added: Lorry ID:L001 load:2500 size:large
=== Cab Service System (Task 01 Full) ===
Demo linked list of passengers: LinkedList(['Alice', 'Bob'])

Options:
 1) Add vehicle
 2) Remove vehicle
 3) Assign (hire) vehicle
 4) Release vehicle
 5) Show available vehicles
 6) Show assigned vehicles
 7) Show bookings history
 8) Exit
Enter choice: 1
Type (Car/Van/ThreeWheeler/Truck/Lorry): Truck
Vehicle ID: 100050
Size ['12ft', '7ft']: 12ft
Added: Truck ID:100050 size:12ft

Options:
 1) Add vehicle
 2) Remove vehicle
 3) Assign (hire) vehicle
 4) Release vehicle
 5) Show available vehicles
 6) Show assigned vehicles
 7) Show bookings history
 8) Exit
Enter choice: 5

Available Cars: (2)
  - Car ID:C001 cap:4 AC:True
  - Car ID:C002 cap:3 AC:False

Available Lorrys: (1)
  - Lorry ID:L001 load:2500 size:large

Available ThreeWheelers: (1)
  - 3W I

In [ ]:
# vehicle_classes.py
%%writefile vehicle_classes.py

from typing import Dict, Any, Optional

class Vehicle:
    def __init__(self, vehicle_id: str, vehicle_type: str):
        self.vehicle_id = vehicle_id
        self.vehicle_type = vehicle_type

    def matches(self, criteria: Dict[str, Any]) -> bool:
        return True

    def summary(self) -> str:
        return f"{self.vehicle_type} (ID: {self.vehicle_id})"


class Car(Vehicle):
    def __init__(self, vehicle_id: str, capacity: int, ac: bool):
        super().__init__(vehicle_id, "Car")
        self.capacity = capacity
        self.ac = ac

    def matches(self, criteria: Dict[str, Any]) -> bool:
        req_p = criteria.get("passengers")
        req_ac = criteria.get("ac")
        if req_p is not None and self.capacity < req_p:
            return False
        if req_ac is True and not self.ac:
            return False
        return True

    def summary(self) -> str:
        return f"Car ID:{self.vehicle_id} cap:{self.capacity} AC:{self.ac}"


class Van(Vehicle):
    def __init__(self, vehicle_id: str, capacity: int, ac: bool):
        super().__init__(vehicle_id, "Van")
        self.capacity = capacity
        self.ac = ac

    def matches(self, criteria: Dict[str, Any]) -> bool:
        req_p = criteria.get("passengers")
        req_ac = criteria.get("ac")
        if req_p is not None and self.capacity < req_p:
            return False
        if req_ac is True and not self.ac:
            return False
        return True

    def summary(self) -> str:
        return f"Van ID:{self.vehicle_id} cap:{self.capacity} AC:{self.ac}"


class ThreeWheeler(Vehicle):
    def __init__(self, vehicle_id: str):
        super().__init__(vehicle_id, "ThreeWheeler")
        self.capacity = 3

    def matches(self, criteria: Dict[str, Any]) -> bool:
        req_p = criteria.get("passengers")
        if req_p is not None and req_p > self.capacity:
            return False
        return True

    def summary(self) -> str:
        return f"3W ID:{self.vehicle_id} cap:{self.capacity}"


class Truck(Vehicle):
    def __init__(self, vehicle_id: str, size: str):
        super().__init__(vehicle_id, "Truck")
        self.size = size

    def matches(self, criteria: Dict[str, Any]) -> bool:
        req_size = criteria.get("size")
        if req_size is not None and self.size != req_size:
            return False
        return True

    def summary(self) -> str:
        return f"Truck ID:{self.vehicle_id} size:{self.size}"


class Lorry(Vehicle):
    def __init__(self, vehicle_id: str, max_load: int, size: Optional[str] = None):
        super().__init__(vehicle_id, "Lorry")
        self.max_load = max_load
        self.size = size

    def matches(self, criteria: Dict[str, Any]) -> bool:
        req_load = criteria.get("max_load")
        req_size = criteria.get("size")
        if req_load is not None and self.max_load < req_load:
            return False
        if req_size is not None and self.size is not None and self.size != req_size:
            return False
        return True

    def summary(self) -> str:
        return f"Lorry ID:{self.vehicle_id} load:{self.max_load}{' size:'+self.size if self.size else ''}"


In [ ]:
# cab_service.py
%%writefile cab_service.py

from collections import deque
from typing import Dict, Any, Optional, List
from vehicle_classes import Vehicle

VALID_TYPES = {"Car", "Van", "ThreeWheeler", "Truck", "Lorry"}

class CabService:
    def __init__(self):
        self.available: Dict[str, deque] = {t: deque() for t in VALID_TYPES}
        self.assigned: Dict[str, Vehicle] = {}
        # bookings: list of dicts demonstrating use of lists (arrays)
        self.bookings: List[Dict[str, Any]] = []
        # basic fare rates per km by vehicle type (demonstrates variables & expressions)
        self.fare_rates = {
            "Car": 30.0,
            "Van": 45.0,
            "ThreeWheeler": 20.0,
            "Truck": 80.0,
            "Lorry": 100.0
        }

    def _vehicle_exists(self, vehicle_id: str) -> bool:
        if vehicle_id in self.assigned:
            return True
        for q in self.available.values():
            for v in q:
                if v.vehicle_id == vehicle_id:
                    return True
        return False

    def _find_in_available(self, vehicle_id: str) -> Optional[Vehicle]:
        for q in self.available.values():
            for v in q:
                if v.vehicle_id == vehicle_id:
                    return v
        return None

    def add_vehicle(self, vehicle: Vehicle) -> None:
        if vehicle.vehicle_type not in VALID_TYPES:
            raise ValueError("Invalid vehicle type.")
        if self._vehicle_exists(vehicle.vehicle_id):
            raise ValueError(f"Vehicle ID '{vehicle.vehicle_id}' already exists.")
        self.available[vehicle.vehicle_type].append(vehicle)
        print(f"Added: {vehicle.summary()}")

    def remove_vehicle(self, vehicle_id: str, force: bool = False) -> None:
        v = self._find_in_available(vehicle_id)
        if v:
            self.available[v.vehicle_type].remove(v)
            print(f"Removed (available): {v.summary()}")
            return
        if vehicle_id in self.assigned:
            if not force:
                raise ValueError("Vehicle is currently assigned. Use force=True to remove.")
            v = self.assigned.pop(vehicle_id)
            print(f"Removed (assigned, forced): {v.summary()}")
            return
        raise ValueError("Vehicle ID not found.")

    def assign_vehicle(self, vtype: str, criteria: Dict[str, Any], customer_name: Optional[str]=None, distance_km: Optional[float]=None) -> Vehicle:
        if vtype not in VALID_TYPES:
            raise ValueError("Invalid vehicle type.")
        queue = self.available[vtype]
        for v in list(queue):
            if v.matches(criteria):
                queue.remove(v)
                self.assigned[v.vehicle_id] = v
                # Create a booking record (demonstrates data capture)
                fare = None
                if distance_km is not None:
                    fare = self.calculate_fare(vtype, distance_km)
                booking = {
                    "vehicle_id": v.vehicle_id,
                    "vehicle_type": vtype,
                    "customer": customer_name,
                    "distance_km": distance_km,
                    "fare": fare
                }
                self.bookings.append(booking)
                print(f"Assigned: {v.summary()} | Booking: {booking}")
                return v
        raise ValueError("No available vehicle matching the criteria.")

    def release_vehicle(self, vehicle_id: str) -> None:
        if vehicle_id not in self.assigned:
            raise ValueError("Vehicle is not currently assigned (or unknown).")
        v = self.assigned.pop(vehicle_id)
        self.available[v.vehicle_type].append(v)
        print(f"Released and returned to queue: {v.summary()}")

    def calculate_fare(self, vtype: str, distance_km: float) -> float:
        # simple fare formula: base_rate_per_km * distance + small booking fee
        rate = self.fare_rates.get(vtype, 30.0)
        booking_fee = 50.0 if distance_km > 0 else 0.0
        # expression demonstration
        fare = rate * distance_km + booking_fee
        # round to 2 decimals
        return round(fare, 2)

    def show_available(self) -> None:
        for t in sorted(VALID_TYPES):
            q = self.available[t]
            print(f"\nAvailable {t}s: ({len(q)})")
            if q:
                for v in q:
                    print("  -", v.summary())
            else:
                print("  None")

    def show_assigned(self) -> None:
        print(f"\nAssigned vehicles: ({len(self.assigned)})")
        if self.assigned:
            for v in self.assigned.values():
                print("  -", v.summary())
        else:
            print("  None")

    def show_bookings(self) -> None:
        print(f"\nBookings history: ({len(self.bookings)})")
        if self.bookings:
            for b in self.bookings:
                print("  -", b)
        else:
            print("  None")


In [ ]:
# vehicle_factory.py
%%writefile vehicle_factory.py

from vehicle_classes import Car, Van, ThreeWheeler, Truck, Lorry

CAR_CAPS = {3, 4}
VAN_CAPS = {6, 8}
TRUCK_SIZES = {"7ft", "12ft"}
LORRY_LOADS = {2500, 3500}

def create_car(vehicle_id: str, capacity: int, ac: bool) -> Car:
    if capacity not in CAR_CAPS:
        raise ValueError(f"Car capacity must be one of {sorted(CAR_CAPS)}")
    return Car(vehicle_id, capacity, ac)

def create_van(vehicle_id: str, capacity: int, ac: bool) -> Van:
    if capacity not in VAN_CAPS:
        raise ValueError(f"Van capacity must be one of {sorted(VAN_CAPS)}")
    return Van(vehicle_id, capacity, ac)

def create_threewheeler(vehicle_id: str) -> ThreeWheeler:
    return ThreeWheeler(vehicle_id)

def create_truck(vehicle_id: str, size: str) -> Truck:
    if size not in TRUCK_SIZES:
        raise ValueError(f"Truck size must be one of {sorted(TRUCK_SIZES)}")
    return Truck(vehicle_id, size)

def create_lorry(vehicle_id: str, max_load: int, size=None) -> Lorry:
    if max_load not in LORRY_LOADS:
        raise ValueError(f"Lorry max load must be one of {sorted(LORRY_LOADS)}")
    return Lorry(vehicle_id, max_load, size)


In [ ]:
# data_structures.py
%%writefile main.py

from typing import Optional, Any

class Node:
    def __init__(self, value: Any):
        self.value = value
        self.next: Optional['Node'] = None

class LinkedList:
    def __init__(self):
        self.head: Optional[Node] = None

    def append(self, value: Any) -> None:
        if not self.head:
            self.head = Node(value)
            return
        cur = self.head
        while cur.next:
            cur = cur.next
        cur.next = Node(value)

    def to_list(self):
        out = []
        cur = self.head
        while cur:
            out.append(cur.value)
            cur = cur.next
        return out

    def __repr__(self):
        return f"LinkedList({self.to_list()})"


In [ ]:
# main.py
%%writefile main.py

from cab_service import CabService
from vehicle_factory import (
    create_car,
    create_van,
    create_threewheeler,
    create_truck,
    create_lorry,
    CAR_CAPS,
    VAN_CAPS,
    TRUCK_SIZES,
    LORRY_LOADS,
)
from data_structures import LinkedList

def main_menu():
    service = CabService()
    # Pre-populate some vehicles for convenience
    try:
        service.add_vehicle(create_car("C001", 4, True))
        service.add_vehicle(create_car("C002", 3, False))
        service.add_vehicle(create_van("V001", 6, True))
        service.add_vehicle(create_threewheeler("T001"))
        service.add_vehicle(create_truck("TR1", "7ft"))
        service.add_vehicle(create_lorry("L001", 2500, "large"))
    except Exception as e:
        print("Preload error:", e)

    print("=== Cab Service System (Task 01 Full) ===")

    while True:
        print("\nOptions:")
        print(" 1) Add vehicle")
        print(" 2) Remove vehicle")
        print(" 3) Assign (hire) vehicle")
        print(" 4) Release vehicle")
        print(" 5) Show available vehicles")
        print(" 6) Show assigned vehicles")
        print(" 7) Show bookings history")
        print(" 8) Exit")

        choice = input("Enter choice: ").strip()

        try:
            if choice == "1":
                vtype = input("Type (Car/Van/ThreeWheeler/Truck/Lorry): ").strip()
                vid = input("Vehicle ID: ").strip()
                if vtype.lower() == "car":
                    cap = int(input(f"Capacity {sorted(CAR_CAPS)}: ").strip())
                    ac = input("AC? (yes/no): ").strip().lower() == "yes"
                    service.add_vehicle(create_car(vid, cap, ac))
                elif vtype.lower() == "van":
                    cap = int(input(f"Capacity {sorted(VAN_CAPS)}: ").strip())
                    ac = input("AC? (yes/no): ").strip().lower() == "yes"
                    service.add_vehicle(create_van(vid, cap, ac))
                elif vtype.lower() in ("threewheeler", "3w", "3wheel"):
                    service.add_vehicle(create_threewheeler(vid))
                elif vtype.lower() == "truck":
                    size = input(f"Size {sorted(TRUCK_SIZES)}: ").strip()
                    service.add_vehicle(create_truck(vid, size))
                elif vtype.lower() == "lorry":
                    load = int(input(f"Max load {sorted(LORRY_LOADS)}: ").strip())
                    size = input("Size (optional): ").strip() or None
                    service.add_vehicle(create_lorry(vid, load, size))
                else:
                    print("Invalid vehicle type.")

            elif choice == "2":
                vid = input("Vehicle ID to remove: ").strip()
                force = input("Force removal if assigned? (yes/no): ").strip().lower() == "yes"
                service.remove_vehicle(vid, force)

            elif choice == "3":
                vtype = input("Type to assign (Car/Van/ThreeWheeler/Truck/Lorry): ").strip()
                criteria = {}
                if vtype.lower() in ("car", "van"):
                    p = input("Passengers needed (enter number or leave blank): ").strip()
                    if p:
                        criteria["passengers"] = int(p)
                    ac_req = input("Require AC? (yes/no/skip): ").strip().lower()
                    if ac_req == "yes": criteria["ac"] = True
                    elif ac_req == "no": criteria["ac"] = False
                elif vtype.lower() in ("threewheeler", "3w", "3wheel"):
                    p = input("Passengers needed (leave blank to accept default 3): ").strip()
                    if p: criteria["passengers"] = int(p)
                elif vtype.lower() == "truck":
                    size = input(f"Size {sorted(TRUCK_SIZES)}: ").strip()
                    if size: criteria["size"] = size
                elif vtype.lower() == "lorry":
                    load = input(f"Min load required (enter number or leave blank): ").strip()
                    if load: criteria["max_load"] = int(load)
                    size = input("Size (optional): ").strip()
                    if size: criteria["size"] = size
                else:
                    print("Invalid type.")
                    continue

                customer = input("Customer name (optional): ").strip() or None
                dist = input("Estimated distance in km (optional): ").strip()
                distance_km = float(dist) if dist else None

                normalized = ("ThreeWheeler" if vtype.lower().startswith("three") or vtype.lower().startswith("3w") else
                              "Car" if vtype.lower() == "car" else
                              "Van" if vtype.lower() == "van" else
                              "Truck" if vtype.lower() == "truck" else
                              "Lorry" if vtype.lower() == "lorry" else vtype)
                service.assign_vehicle(normalized, criteria, customer_name=customer, distance_km=distance_km)

            elif choice == "4":
                vid = input("Vehicle ID to release: ").strip()
                service.release_vehicle(vid)

            elif choice == "5":
                service.show_available()

            elif choice == "6":
                service.show_assigned()

            elif choice == "7":
                service.show_bookings()

            elif choice == "8":
                print("Goodbye.")
                break

            else:
                print("Unknown option.")

        except Exception as e:
            print("Error:", e)

if __name__ == "__main__":
    main_menu()

In [ ]:
# This cell writes module files to disk so `main.py` imports work as normal modules.
files = {
    'vehicle_classes.py': "# vehicle_classes.py\nfrom typing import Dict, Any, Optional\n\nclass Vehicle:\n    def __init__(self, vehicle_id: str, vehicle_type: str):\n        self.vehicle_id = vehicle_id\n        self.vehicle_type = vehicle_type\n\n    def matches(self, criteria: Dict[str, Any]) -> bool:\n        return True\n\n    def summary(self) -> str:\n        return f\"{self.vehicle_type} (ID: {self.vehicle_id})\"\n\n\nclass Car(Vehicle):\n    def __init__(self, vehicle_id: str, capacity: int, ac: bool):\n        super().__init__(vehicle_id, \"Car\")\n        self.capacity = capacity\n        self.ac = ac\n\n    def matches(self, criteria: Dict[str, Any]) -> bool:\n        req_p = criteria.get(\"passengers\")\n        req_ac = criteria.get(\"ac\")\n        if req_p is not None and self.capacity < req_p:\n            return False\n        if req_ac is True and not self.ac:\n            return False\n        return True\n\n    def summary(self) -> str:\n        return f\"Car ID:{self.vehicle_id} cap:{self.capacity} AC:{self.ac}\"\n\n\nclass Van(Vehicle):\n    def __init__(self, vehicle_id: str, capacity: int, ac: bool):\n        super().__init__(vehicle_id, \"Van\")\n        self.capacity = capacity\n        self.ac = ac\n\n    def matches(self, criteria: Dict[str, Any]) -> bool:\n        req_p = criteria.get(\"passengers\")\n        req_ac = criteria.get(\"ac\")\n        if req_p is not None and self.capacity < req_p:\n            return False\n        if req_ac is True and not self.ac:\n            return False\n        return True\n\n    def summary(self) -> str:\n        return f\"Van ID:{self.vehicle_id} cap:{self.capacity} AC:{self.ac}\"\n\n\nclass ThreeWheeler(Vehicle):\n    def __init__(self, vehicle_id: str):\n        super().__init__(vehicle_id, \"ThreeWheeler\")\n        self.capacity = 3\n\n    def matches(self, criteria: Dict[str, Any]) -> bool:\n        req_p = criteria.get(\"passengers\")\n        if req_p is not None and req_p > self.capacity:\n            return False\n        return True\n\n    def summary(self) -> str:\n        return f\"3W ID:{self.vehicle_id} cap:{self.capacity}\"\n\n\nclass Truck(Vehicle):\n    def __init__(self, vehicle_id: str, size: str):\n        super().__init__(vehicle_id, \"Truck\")\n        self.size = size\n\n    def matches(self, criteria: Dict[str, Any]) -> bool:\n        req_size = criteria.get(\"size\")\n        if req_size is not None and self.size != req_size:\n            return False\n        return True\n\n    def summary(self) -> str:\n        return f\"Truck ID:{self.vehicle_id} size:{self.size}\"\n\n\nclass Lorry(Vehicle):\n    def __init__(self, vehicle_id: str, max_load: int, size: Optional[str] = None):\n        super().__init__(vehicle_id, \"Lorry\")\n        self.max_load = max_load\n        self.size = size\n\n    def matches(self, criteria: Dict[str, Any]) -> bool:\n        req_load = criteria.get(\"max_load\")\n        req_size = criteria.get(\"size\")\n        if req_load is not None and self.max_load < req_load:\n            return False\n        if req_size is not None and self.size is not None and self.size != req_size:\n            return False\n        return True\n\n    def summary(self) -> str:\n        return f\"Lorry ID:{self.vehicle_id} load:{self.max_load}{' size:'+self.size if self.size else ''}\"\n",
    'cab_service.py': "# cab_service.py\nfrom collections import deque\nfrom typing import Dict, Any, Optional, List\nfrom vehicle_classes import Vehicle\n\nVALID_TYPES = {\"Car\", \"Van\", \"ThreeWheeler\", \"Truck\", \"Lorry\"}\n\nclass CabService:\n    def __init__(self):\n        self.available: Dict[str, deque] = {t: deque() for t in VALID_TYPES}\n        self.assigned: Dict[str, Vehicle] = {}\n        # bookings: list of dicts demonstrating use of lists (arrays)\n        self.bookings: List[Dict[str, Any]] = []\n        # basic fare rates per km by vehicle type (demonstrates variables & expressions)\n        self.fare_rates = {\n            \"Car\": 30.0,\n            \"Van\": 45.0,\n            \"ThreeWheeler\": 20.0,\n            \"Truck\": 80.0,\n            \"Lorry\": 100.0\n        }\n\n    def _vehicle_exists(self, vehicle_id: str) -> bool:\n        if vehicle_id in self.assigned:\n            return True\n        for q in self.available.values():\n            for v in q:\n                if v.vehicle_id == vehicle_id:\n                    return True\n        return False\n\n    def _find_in_available(self, vehicle_id: str) -> Optional[Vehicle]:\n        for q in self.available.values():\n            for v in q:\n                if v.vehicle_id == vehicle_id:\n                    return v\n        return None\n\n    def add_vehicle(self, vehicle: Vehicle) -> None:\n        if vehicle.vehicle_type not in VALID_TYPES:\n            raise ValueError(\"Invalid vehicle type.\")\n        if self._vehicle_exists(vehicle.vehicle_id):\n            raise ValueError(f\"Vehicle ID '{vehicle.vehicle_id}' already exists.\")\n        self.available[vehicle.vehicle_type].append(vehicle)\n        print(f\"Added: {vehicle.summary()}\")\n\n    def remove_vehicle(self, vehicle_id: str, force: bool = False) -> None:\n        v = self._find_in_available(vehicle_id)\n        if v:\n            self.available[v.vehicle_type].remove(v)\n            print(f\"Removed (available): {v.summary()}\")\n            return\n        if vehicle_id in self.assigned:\n            if not force:\n                raise ValueError(\"Vehicle is currently assigned. Use force=True to remove.\")\n            v = self.assigned.pop(vehicle_id)\n            print(f\"Removed (assigned, forced): {v.summary()}\")\n            return\n        raise ValueError(\"Vehicle ID not found.\")\n\n    def assign_vehicle(self, vtype: str, criteria: Dict[str, Any], customer_name: Optional[str]=None, distance_km: Optional[float]=None) -> Vehicle:\n        if vtype not in VALID_TYPES:\n            raise ValueError(\"Invalid vehicle type.\")\n        queue = self.available[vtype]\n        for v in list(queue):\n            if v.matches(criteria):\n                queue.remove(v)\n                self.assigned[v.vehicle_id] = v\n                # Create a booking record (demonstrates data capture)\n                fare = None\n                if distance_km is not None:\n                    fare = self.calculate_fare(vtype, distance_km)\n                booking = {\n                    \"vehicle_id\": v.vehicle_id,\n                    \"vehicle_type\": vtype,\n                    \"customer\": customer_name,\n                    \"distance_km\": distance_km,\n                    \"fare\": fare\n                }\n                self.bookings.append(booking)\n                print(f\"Assigned: {v.summary()} | Booking: {booking}\")\n                return v\n        raise ValueError(\"No available vehicle matching the criteria.\")\n\n    def release_vehicle(self, vehicle_id: str) -> None:\n        if vehicle_id not in self.assigned:\n            raise ValueError(\"Vehicle is not currently assigned (or unknown).\")\n        v = self.assigned.pop(vehicle_id)\n        self.available[v.vehicle_type].append(v)\n        print(f\"Released and returned to queue: {v.summary()}\")\n\n    def calculate_fare(self, vtype: str, distance_km: float) -> float:\n        # simple fare formula: base_rate_per_km * distance + small booking fee\n        rate = self.fare_rates.get(vtype, 30.0)\n        booking_fee = 50.0 if distance_km > 0 else 0.0\n        # expression demonstration\n        fare = rate * distance_km + booking_fee\n        # round to 2 decimals\n        return round(fare, 2)\n\n    def show_available(self) -> None:\n        for t in sorted(VALID_TYPES):\n            q = self.available[t]\n            print(f\"\\nAvailable {t}s: ({len(q)})\")\n            if q:\n                for v in q:\n                    print(\"  -\", v.summary())\n            else:\n                print(\"  None\")\n\n    def show_assigned(self) -> None:\n        print(f\"\\nAssigned vehicles: ({len(self.assigned)})\")\n        if self.assigned:\n            for v in self.assigned.values():\n                print(\"  -\", v.summary())\n        else:\n            print(\"  None\")\n\n    def show_bookings(self) -> None:\n        print(f\"\\nBookings history: ({len(self.bookings)})\")\n        if self.bookings:\n            for b in self.bookings:\n                print(\"  -\", b)\n        else:\n            print(\"  None\")\n",
    'vehicle_factory.py': "# vehicle_factory.py\nfrom vehicle_classes import Car, Van, ThreeWheeler, Truck, Lorry\n\nCAR_CAPS = {3, 4}\nVAN_CAPS = {6, 8}\nTRUCK_SIZES = {\"7ft\", \"12ft\"}\nLORRY_LOADS = {2500, 3500}\n\ndef create_car(vehicle_id: str, capacity: int, ac: bool) -> Car:\n    if capacity not in CAR_CAPS:\n        raise ValueError(f\"Car capacity must be one of {sorted(CAR_CAPS)}\")\n    return Car(vehicle_id, capacity, ac)\n\ndef create_van(vehicle_id: str, capacity: int, ac: bool) -> Van:\n    if capacity not in VAN_CAPS:\n        raise ValueError(f\"Van capacity must be one of {sorted(VAN_CAPS)}\")\n    return Van(vehicle_id, capacity, ac)\n\ndef create_threewheeler(vehicle_id: str) -> ThreeWheeler:\n    return ThreeWheeler(vehicle_id)\n\ndef create_truck(vehicle_id: str, size: str) -> Truck:\n    if size not in TRUCK_SIZES:\n        raise ValueError(f\"Truck size must be one of {sorted(TRUCK_SIZES)}\")\n    return Truck(vehicle_id, size)\n\ndef create_lorry(vehicle_id: str, max_load: int, size=None) -> Lorry:\n    if max_load not in LORRY_LOADS:\n        raise ValueError(f\"Lorry max load must be one of {sorted(LORRY_LOADS)}\")\n    return Lorry(vehicle_id, max_load, size)\n",
    'data_structures.py': "# data_structures.py\n# Demonstrates a simple singly linked list and usage alongside Python lists.\nfrom typing import Optional, Any\n\nclass Node:\n    def __init__(self, value: Any):\n        self.value = value\n        self.next: Optional['Node'] = None\n\nclass LinkedList:\n    def __init__(self):\n        self.head: Optional[Node] = None\n\n    def append(self, value: Any) -> None:\n        if not self.head:\n            self.head = Node(value)\n            return\n        cur = self.head\n        while cur.next:\n            cur = cur.next\n        cur.next = Node(value)\n\n    def to_list(self):\n        out = []\n        cur = self.head\n        while cur:\n            out.append(cur.value)\n            cur = cur.next\n        return out\n\n    def __repr__(self):\n        return f\"LinkedList({self.to_list()})\"\n",
    'main.py': "# main.py\nfrom cab_service import CabService\nfrom vehicle_factory import (\n    create_car,\n    create_van,\n    create_threewheeler,\n    create_truck,\n    create_lorry,\n    CAR_CAPS,\n    VAN_CAPS,\n    TRUCK_SIZES,\n    LORRY_LOADS,\n)\nfrom data_structures import LinkedList\n\ndef main_menu():\n    service = CabService()\n    # Pre-populate some vehicles for convenience\n    try:\n        service.add_vehicle(create_car(\"C001\", 4, True))\n        service.add_vehicle(create_car(\"C002\", 3, False))\n        service.add_vehicle(create_van(\"V001\", 6, True))\n        service.add_vehicle(create_threewheeler(\"T001\"))\n        service.add_vehicle(create_truck(\"TR1\", \"7ft\"))\n        service.add_vehicle(create_lorry(\"L001\", 2500, \"large\"))\n    except Exception as e:\n        print(\"Preload error:\", e)\n\n    print(\"=== Cab Service System (Task 01 Full) ===\")\n    # Example LinkedList usage (demonstrates data structures)\n    passenger_list = LinkedList()\n    passenger_list.append(\"Alice\")\n    passenger_list.append(\"Bob\")\n    print(\"Demo linked list of passengers:\", passenger_list)\n\n    while True:\n        print(\"\\nOptions:\")\n        print(\" 1) Add vehicle\")\n        print(\" 2) Remove vehicle\")\n        print(\" 3) Assign (hire) vehicle\")\n        print(\" 4) Release vehicle\")\n        print(\" 5) Show available vehicles\")\n        print(\" 6) Show assigned vehicles\")\n        print(\" 7) Show bookings history\")\n        print(\" 8) Exit\")\n\n        choice = input(\"Enter choice: \").strip()\n\n        try:\n            if choice == \"1\":\n                vtype = input(\"Type (Car/Van/ThreeWheeler/Truck/Lorry): \").strip()\n                vid = input(\"Vehicle ID: \").strip()\n                if vtype.lower() == \"car\":\n                    cap = int(input(f\"Capacity {sorted(CAR_CAPS)}: \").strip())\n                    ac = input(\"AC? (yes/no): \").strip().lower() == \"yes\"\n                    service.add_vehicle(create_car(vid, cap, ac))\n                elif vtype.lower() == \"van\":\n                    cap = int(input(f\"Capacity {sorted(VAN_CAPS)}: \").strip())\n                    ac = input(\"AC? (yes/no): \").strip().lower() == \"yes\"\n                    service.add_vehicle(create_van(vid, cap, ac))\n                elif vtype.lower() in (\"threewheeler\", \"3w\", \"3wheel\"):\n                    service.add_vehicle(create_threewheeler(vid))\n                elif vtype.lower() == \"truck\":\n                    size = input(f\"Size {sorted(TRUCK_SIZES)}: \").strip()\n                    service.add_vehicle(create_truck(vid, size))\n                elif vtype.lower() == \"lorry\":\n                    load = int(input(f\"Max load {sorted(LORRY_LOADS)}: \").strip())\n                    size = input(\"Size (optional): \").strip() or None\n                    service.add_vehicle(create_lorry(vid, load, size))\n                else:\n                    print(\"Invalid vehicle type.\")\n\n            elif choice == \"2\":\n                vid = input(\"Vehicle ID to remove: \").strip()\n                force = input(\"Force removal if assigned? (yes/no): \").strip().lower() == \"yes\"\n                service.remove_vehicle(vid, force)\n\n            elif choice == \"3\":\n                vtype = input(\"Type to assign (Car/Van/ThreeWheeler/Truck/Lorry): \").strip()\n                criteria = {}\n                if vtype.lower() in (\"car\", \"van\"):\n                    p = input(\"Passengers needed (enter number or leave blank): \").strip()\n                    if p:\n                        criteria[\"passengers\"] = int(p)\n                    ac_req = input(\"Require AC? (yes/no/skip): \").strip().lower()\n                    if ac_req == \"yes\": criteria[\"ac\"] = True\n                    elif ac_req == \"no\": criteria[\"ac\"] = False\n                elif vtype.lower() in (\"threewheeler\", \"3w\", \"3wheel\"):\n                    p = input(\"Passengers needed (leave blank to accept default 3): \").strip()\n                    if p: criteria[\"passengers\"] = int(p)\n                elif vtype.lower() == \"truck\":\n                    size = input(f\"Size {sorted(TRUCK_SIZES)}: \").strip()\n                    if size: criteria[\"size\"] = size\n                elif vtype.lower() == \"lorry\":\n                    load = input(f\"Min load required (enter number or leave blank): \").strip()\n                    if load: criteria[\"max_load\"] = int(load)\n                    size = input(\"Size (optional): \").strip()\n                    if size: criteria[\"size\"] = size\n                else:\n                    print(\"Invalid type.\")\n                    continue\n\n                customer = input(\"Customer name (optional): \").strip() or None\n                dist = input(\"Estimated distance in km (optional): \").strip()\n                distance_km = float(dist) if dist else None\n\n                normalized = (\"ThreeWheeler\" if vtype.lower().startswith(\"three\") or vtype.lower().startswith(\"3w\") else\n                              \"Car\" if vtype.lower() == \"car\" else\n                              \"Van\" if vtype.lower() == \"van\" else\n                              \"Truck\" if vtype.lower() == \"truck\" else\n                              \"Lorry\" if vtype.lower() == \"lorry\" else vtype)\n                service.assign_vehicle(normalized, criteria, customer_name=customer, distance_km=distance_km)\n\n            elif choice == \"4\":\n                vid = input(\"Vehicle ID to release: \").strip()\n                service.release_vehicle(vid)\n\n            elif choice == \"5\":\n                service.show_available()\n\n            elif choice == \"6\":\n                service.show_assigned()\n\n            elif choice == \"7\":\n                service.show_bookings()\n\n            elif choice == \"8\":\n                print(\"Goodbye.\")\n                break\n\n            else:\n                print(\"Unknown option.\")\n\n        except Exception as e:\n            print(\"Error:\", e)\n\nif __name__ == \"__main__\":\n    main_menu()\n",
}

for name, content in files.items():
    with open(name, 'w', encoding='utf-8') as f:
        f.write(content)
print('Wrote files:', list(files.keys()))
